# Подготовка

   Ниже закомментированный код, который помог мне переименовать изображения и подготовить датасет. Запускать его не следует, датасет будет в архиве.
  Я отложил 100 изображений (50 часов и 50 крокодилов) для теста. Остальные 900 поворачиваем 3 раза, и делаем зеркальное отражение для каждой получившейся. Таким обазом, мы расширяем датасет, на котором обучаемся (теперь 7200 изображений) 

In [29]:
#import numpy as np
#import os
#from PIL import Image, ImageDraw  
#path = os.getcwd()
#for u in np.arange(551,1001):
 #   os.chdir(path)
  #  os.rename(str(u)+'.png', str(u-100)+'.png')
#for u in np.arange(900,0,-1):
#    os.chdir(path)
#    os.rename(str(u)+'.png', str(u*8)+'.png')

#for i in range(1,1001):
#    os.chdir(path)
#    image=Image.open(str(i*8)+'.png')
#    im1=image.rotate(90)
#    im1.save(str(i*8-7)+'.png')
#    im2=im1.rotate(90)
#    im2.save(str(i*8-6)+'.png')
#    im3=im2.rotate(90)
#    im3.save(str(i*8-5)+'.png')        
#    im4=image.transpose(Image.FLIP_LEFT_RIGHT)
#    im4.save(str(i*8-4)+'.png')
#    im5=im1.transpose(Image.FLIP_LEFT_RIGHT)
#    im5.save(str(i*8-3)+'.png')
#    im6=im2.transpose(Image.FLIP_LEFT_RIGHT)
#    im6.save(str(i*8-2)+'.png')
#    im7=im3.transpose(Image.FLIP_LEFT_RIGHT)
#    im7.save(str(i*8-1)+'.png')


В качестве признаков я использовал:

 1.Сумму по R, G, B каналам (3 нормированных числа)

2.Гистограмма от черно-белой картинки. Она представляет из себя длинный вектор, который я разбиваю на 4 части и в каждой из частей усредняю числа. Аналогично для гистограмм от R, G, B каналов от цветной картинки. (В итоге 4 гистограммы)

3.Вектор, который выдает метод HOG (библиотека skimage), гистограммы направленных градиентов

In [172]:
import pandas as pd
import numpy as np
import math
import random
import os
from PIL import Image, ImageDraw  
from sklearn import cross_validation, linear_model, metrics, svm
from sklearn.utils import shuffle
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score, train_test_split
from skimage import feature
from sklearn.metrics import roc_auc_score

def histo(histogram):
    k=0
    f1=[]
    for i in range(0,4):
        summ=0
        for j in range(k,k+64):
            summ=summ+histogram[k]
            k=k+1
        f1.append(float(summ)/64)
    return f1



### Создание выборки

In [173]:
features=range(0,343)
data=pd.DataFrame(columns=features)
target=[0]*3600+[1]*3600+[0]*50+[1]*50
for t in range(1,7301):
    image = Image.open(str(t)+'.png') 
    width = image.size[0] 
    height = image.size[1]
    pix=image.load()
    R,G,B=0,0,0
    for i in range(width):
        for j in range(height):
            R = R + pix[i, j][0]
            G = G + pix[i, j][1]
            B = B + pix[i, j][2]
    S=R+G+B
    R=round(float(R)/S,3)
    G=round(float(G)/S,3)
    B=round(float(B)/S,3) 
    ch_b=image.convert(mode="L")
    hist_ch_b = ch_b.histogram()
    k=0
    hist=image.histogram()
    f=histo(hist_ch_b)
    hist1=hist[0:256]
    hist2=hist[256:512]
    hist3=hist[512:768]
    f1=histo(hist1)
    f2=histo(hist2)
    f3=histo(hist3)
    I = np.asarray(ch_b)
    hog=feature.hog(I,orientations=9,feature_vector=True)
    data.loc[t]=np.hstack(([R,G,B],f,f1,f2,f3,hog))
data['target']=target

### Масштабирование выборки

In [174]:
X = scale(data[data.columns[:-1]])


### Разбиение на трейн и тест

In [175]:
X_train1=X[0:7200]
X_test=X[7200:]
y_train1=target[0:7200]
y_test=target[7200:]


A=pd.DataFrame(X_train1)
A['target']=y_train1
new_A=shuffle(A)
X_train=new_A[new_A.columns[:-1]].as_matrix()
y_train=np.asarray(new_A[new_A.columns[-1:]].as_matrix()).reshape(1,-1)[0]

# Обучение

В качестве модели использую метод опорных векторов (SVM), с предсказательной способностью. (probability=True)

Подберем лучшие параметры на кросс-валидации. Варьирую C (штраф)

In [96]:
#a=[[]]
#maxim=0.01
#maxi=0
#for i in np.arange(0.3, 2, 0.1):
#    R = svm.SVC(C=i, probability=True)
#    scores = cross_val_score(R, X_train, y_train, cv=3)
#    cur=scores.mean()
#    if cur>maxim:
#        maxim=scores.mean()
#        maxi=i


            
            
#print maxi, maxim

1.4000000000000004 0.9454166666666666


Возьмем С=1.5. Этот кусок считается очень долго, поэтому совсем честно проверить не успел. Возможно нужны значения побольше и варьировать другие параметры. Accuracy получился 0.945

Обучаем машину и смотрим accuracy на тесте

In [176]:
my_svm=svm.SVC(C=1.5, probability=True)
my_svm.fit(X_train,y_train)
predictions = my_svm.predict(X_test)

proba_predictions = my_svm.predict_proba(X_test)
print 'accuracy = ', metrics.accuracy_score(y_test, predictions)

accuracy =  0.93


На тестовой выборке accuracy=0.93

# Похожие картинки

Попросим обученный классификатор найти картинки, похожие и на часы, и на крокодила. У таких объектов вероятности принадлежности к классу близки к 0.5. Сначала подготовим базу данных без повернутых и отраженных картинок. 

In [177]:
data_1000=pd.DataFrame(columns=features)
for i in range(0,900):
    data_1000.loc[i]=data.loc[(i+1)*8]
for i in range(900,1000):
    data_1000.loc[i]=data.loc[i+6301]
    

X_1000=np.vstack((X_train1[7:7201:8],X_test))
y_1000=np.hstack((y_train[7:7201:8],y_test))


Теперь посмотрим, вероятности каких объектов лежат в пределах (0.44 ; 0.56), то есть близки к 0.5, что означает неуверенность машины.

In [183]:
massiv=[]
proba_predictions = my_svm.predict_proba(X_1000)
e=1
for r in proba_predictions:
        if abs(r[0]-r[1])<0.12:
            massiv.append(e)
        e=e+1
print massiv

[258, 294, 379, 903, 939]


Выдает множество: [258, 294, 379, 903, 939]. Если перевести это на номера картинок в датасете(7300), то это:


2072.png, 2352.png, 3032.png, 7204.png, 7240.png

На 2072.png и 3032.png, как мне кажется, действительно неочевидно. Мы можем понять, что это часы только благодаря циферблату.  

Мне кажется, что результат можно улучшить. Во-первых, я не успел разобраться в том, как работает HOG, может быть надо было варьировать его параметры и сильно укоротить выдаваемый вектор. Так же, надо лучше поработать с набором обычных признаков, возможно нецелесообразно добавлять гистограммы в том виде, в котором делал я. Во-вторых, здесь могли лучше сработать методы, которые используют словарь визуальных слов или выделяют геометрические формы, это помогло бы для отделения часов.